In [ ]:
import requests
import pandas as pd
import time
import os
import openpyxl

def fetch_data_and_save_to_excel():
    url = 'https://www.shipxy.com/ship/GetShip'
#     Custom Headers
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': 'xxx',
        'Origin': 'https://www.shipxy.com',
        'Referer': 'https://www.shipxy.com/Ship/Index',
        'User-Agent': 'xxx',
        'X-Requested-With': 'XMLHttpRequest'
    }
    
    payload = {
        'mmsi': '567999111'
    }

    response = requests.post(url, headers=headers, data=payload)

    try:
        response.raise_for_status()  # Check if the request was successful
        data = response.json()
        if 'data' in data:
            df = pd.DataFrame(data['data'])
            file_name = 'ship_data.xlsx'
            if not os.path.isfile(file_name):
                df.to_excel(file_name, index=True, sheet_name='Sheet1')
            else:
                book = openpyxl.load_workbook(file_name)
                writer = pd.ExcelWriter(file_name, engine='openpyxl')
                writer.book = book
                writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

                sheet_name = 'Sheet1'
                if sheet_name in writer.sheets:
                    startrow = writer.sheets[sheet_name].max_row
                    df.to_excel(writer, index=True, header=False, sheet_name=sheet_name, startrow=startrow)
                else:
                    df.to_excel(writer, index=True, header=False, sheet_name=sheet_name)

                writer.save()
                writer.close()

            print("Data saved successfully!")
            return True
        else:
            print("No data found in the response.", data) 
            print("Response content:", response.content)
    except requests.exceptions.HTTPError as err:
        print("HTTP error occurred:", err)
    except Exception as err:
        print("An error occurred:", err)
        
    return False

# Define the number of iterations
num_iterations = 5
failed_attempts = 0

for i in range(num_iterations):
    success = fetch_data_and_save_to_excel()
    if not success:
        failed_attempts += 1

    time.sleep(1)  # Add a 1-second delay between each iteration

print("Total failed attempts:", failed_attempts)